https://github.com/cvg/Hierarchical-Localization  

In [ ]:
# #@title ~ Prepare env
# from google.colab import drive
# drive.mount('/content/gdrive')
%cd '/home/code/Hierarchical-Localization'
!pip install -q -r requirements.txt
!git submodule update --init --recursive
!pip install -U pycolmap

/home/code/Hierarchical-Localization


In [ ]:
#@title 1 Import all python packages
import os, glob, random, shutil, pickle, warnings
from tqdm import tqdm
from pathlib import Path
import numpy as np
import pandas as pd

import cv2
from PIL import Image
from skimage.restoration import inpaint_biharmonic
from skimage.filters import threshold_yen, threshold_otsu, median, threshold_local
from skimage.morphology import remove_small_objects, disk, square, binary_erosion, binary_dilation, binary_closing, erosion, dilation, opening, closing, skeletonize
from skimage.measure import label, regionprops, regionprops_table
from scipy.ndimage import distance_transform_edt, generic_filter, binary_hit_or_miss
from skimage.segmentation import watershed
from skimage.feature import peak_local_max, canny
from skimage.filters import threshold_minimum
from skimage.transform import probabilistic_hough_line
from skimage.color import convert_colorspace
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import pycolmap
from hloc.utils.viz import (plot_images, plot_keypoints, plot_matches, cm_RdGn, add_text)
from hloc.utils.io import read_image
from hloc import extract_features, match_features, reconstruction, visualization, pairs_from_exhaustive, pairs_from_retrieval
from hloc.visualization import plot_images, read_image
from hloc.utils import viz_3d, viz
from hloc import extract_features, match_features, reconstruction, visualization#, pairs_from_retrieval
from hloc.my_module import visualize_sfm_2d

warnings.filterwarnings("ignore")  # 忽略所有 Warning

In [ ]:
#@title 2 Load test data table
data_root = "/home/data"
df = pd.read_csv(f"{data_root}/test_data_after_pred_bbox_back.csv", usecols=['image_file','bbox1'])
df = df.dropna()
df['dir_name'] = df['image_file'].map(lambda x: "/".join(x.split('/')[:-1]))
df

,image_file,bbox1,dir_name
0,C_001/C_001_01/00000001.png,715:399:1023:733,C_001/C_001_01
1,C_001/C_001_01/00000002.png,710:372:1038:738,C_001/C_001_01
2,C_001/C_001_01/00000003.png,718:393:1017:724,C_001/C_001_01
3,C_001/C_001_01/00000005.png,719:374:1035:711,C_001/C_001_01
4,C_001/C_001_01/00000006.png,724:369:1028:705,C_001/C_001_01
...,...,...,...
6456,GI_003/GI_003_wire/00002269.png,592:117:1278:960,GI_003/GI_003_wire
6460,GI_003/GI_003_wire/00003538.png,426:253:574:397,GI_003/GI_003_wire
6462,GI_003/GI_003_wire/00004551.png,421:492:644:742,GI_003/GI_003_wire
6518,GI_004_duodenal polyp and pyloric ring/GI_004_...,942:465:1271:959,GI_004_duodenal polyp and pyloric ring/GI_004_01


In [ ]:
#@title 3 make functions
def remove_glare_v1(image):
    # normalize
    image = image / image.max()
    scale = image.shape[1] / 1280
    radius1 = int(3 * scale)
    radius2 = int(5 * scale)
    # threshold
    YIQ = convert_colorspace(image, 'RGB', 'YIQ')
    high_mask = YIQ[:,:,0] > 0.85
    glare_mask = YIQ[:,:,0] > 0.90
    # reduce Luma
    YIQ[:,:,0] = cv2.inpaint((YIQ[:,:,0]*255).astype('uint8'), binary_dilation(high_mask, disk(radius1, dtype=bool)).astype('uint8'), inpaintRadius=radius1, flags=cv2.INPAINT_TELEA) / 255
    RGB = convert_colorspace(YIQ, 'YIQ', 'RGB')
    RGB = np.clip(RGB * 255, a_min=0, a_max=255).astype('uint8')
    # remove glare
    RGB2 = cv2.inpaint(RGB, binary_dilation(glare_mask, disk(radius2, dtype=bool)).astype('uint8'), inpaintRadius=radius1, flags=cv2.INPAINT_TELEA)
    return RGB2 # [0-255]

def visualize_sfm_2d(reconstruction, image_dir, color_by='visibility', selected=[], n=1, seed=0, dpi=75):
    assert image_dir.exists()
    if not isinstance(reconstruction, pycolmap.Reconstruction):
        reconstruction = pycolmap.Reconstruction(reconstruction)

    if not selected:
        image_ids = reconstruction.reg_image_ids()
        # selected = random.Random(seed).sample(
        #         image_ids, min(n, len(image_ids)))
    result = []
    # for i in selected:
    
    for i in image_ids:
        image = reconstruction.images[i]
        keypoints = np.array([p.xy for p in image.points2D])
        visible = np.array([p.has_point3D() for p in image.points2D])

        if color_by == 'visibility':
            color = [(0, 0, 1) if v else (1, 0, 0) for v in visible]
            text = f'visible: {np.count_nonzero(visible)}/{len(visible)}'
        elif color_by == 'track_length':
            tl = np.array([reconstruction.points3D[p.point3D_id].track.length()
                            if p.has_point3D() else 1 for p in image.points2D])
            max_, med_ = np.max(tl), np.median(tl[tl > 1])
            tl = np.log(tl)
            color = cm.jet(tl / tl.max()).tolist()
            text = f'max/median track length: {max_}/{med_}'
        elif color_by == 'depth':
            p3ids = [p.point3D_id for p in image.points2D if p.has_point3D()]
            z = np.array([image.transform_to_image(
                reconstruction.points3D[j].xyz)[-1] for j in p3ids])
            # z -= z.min()
            color = cm.jet(z / np.percentile(z, 99.9))
            text = f'visible: {np.count_nonzero(visible)}/{len(visible)}'
            keypoints = keypoints[visible]
        else:
            raise NotImplementedError(f'Coloring not implemented: {color_by}.')

        name = image.name
        # plot_images([read_image(image_dir / name)], dpi=dpi)
        # plot_keypoints([keypoints], colors=[color], ps=4)
        # add_text(0, text)
        # add_text(0, name, pos=(0.01, 0.01), fs=5, lcolor=None, va='bottom')
        # result_list = result_list.append([float(keypoints),image.name])
        tmplist = []
        tmp = np.append(keypoints, np.expand_dims(z, axis=1), axis=1)
        tmplist = tmp.tolist()
        tmplist.append([image.name])
        result.append(tmplist)
    return result  # return result_list

In [ ]:
#@title 4 List and selet folders
all_img_dir = sorted([path for path in glob.glob(f"{data_root}/**/*") if ('wire' not in path)and('.zip' not in path)])
select_folders = all_img_dir[:]
select_folders

['/home/data/C_001/C_001_01',
 '/home/data/C_001/C_001_02',
 '/home/data/C_001/C_001_03',
 '/home/data/C_001/C_001_04',
 '/home/data/C_002/C_002_01',
 '/home/data/C_002/C_002_02',
 '/home/data/C_002/C_002_03',
 '/home/data/C_002/C_002_04',
 '/home/data/C_002/C_002_05',
 '/home/data/C_003/C_003_01',
 '/home/data/C_003/C_003_02',
 '/home/data/C_003/C_003_03',
 '/home/data/C_003/C_003_04',
 '/home/data/C_003/C_003_05',
 '/home/data/C_003/C_003_06',
 '/home/data/C_003/C_003_07',
 '/home/data/C_004_T colon polyp/C_004_01',
 '/home/data/C_006_cecal seed/C_006_01',
 '/home/data/C_006_cecal seed/C_006_02',
 '/home/data/C_006_cecal seed/C_006_03',
 '/home/data/C_006_cecal seed/C_006_04',
 '/home/data/C_006_cecal seed/C_006_05',
 '/home/data/C_006_cecal seed/C_006_06',
 '/home/data/C_006_cecal seed/C_006_07',
 '/home/data/GI_001/GI_001_01',
 '/home/data/GI_002/GI_002_01',
 '/home/data/GI_002/GI_002_02',
 '/home/data/GI_002/GI_002_03',
 '/home/data/GI_004_duodenal polyp and pyloric ring/GI_004_01

In [ ]:
#@title 5 hloc method configuration
retrieval_conf = extract_features.confs['netvlad']
feature_conf = extract_features.confs['superpoint_aachen']  # [superpoint_aachen, d2net-ss, r2d2, sift ...]
matcher_conf = match_features.confs['superglue']  # [superglue, adalam ...]

## camera_model: RADIAL_FISHEYE  # camera_params  # https://github.com/cvg/Hierarchical-Localization/pull/210
f, cx, cy, k1, k2 = (645.569196, 640, 480, -0.167759, 0.009857)
# reader_options = dict(camera_model='RADIAL_FISHEYE', camera_params=','.join(map(str, (f, cx, cy, k1, k2))))
reader_options = dict(camera_model='SIMPLE_RADIAL_FISHEYE', camera_params=','.join(map(str, ([295.47721773, 188.74051961, 166.47973187,  -0.41604671]))))
reader_options

{'camera_model': 'SIMPLE_RADIAL_FISHEYE',
 'camera_params': '295.47721773,188.74051961,166.47973187,-0.41604671'}

In [ ]:
#@title 6 Process all images in selected folder
output_root = "/home/test_outdoor"  # do not change !!
is_glare_remove = True
scale = 0.5
roi_ratio = 0.8
is_find_left_right_points = True
magnification = 1.1

for img_dir in select_folders:
    dir_name = "/".join(img_dir.split('/')[-2:])
    print(f"[Start] ======================================== {dir_name} ==========================================")

    # choice data
    max_file_num = 50
    indexes = df[df['dir_name']==dir_name].index.values
    if len(indexes) > max_file_num:
        indexes = np.sort(np.random.choice(indexes, size=max_file_num, replace=False))
    image_file_list = df.loc[indexes, 'image_file'].values

    # Path settings
    input_dir = Path(f"{output_root}/Image_Preprocessed")
    output_dir = Path(f"{output_root}/sfm_outputs")
    result_dir = f"{output_root}/overlap_result_image"
    sfm_pairs = output_dir / 'pairs-netvlad.txt'
    loc_pairs = output_dir / 'pairs-loc.txt'
    sfm_dir = output_dir / 'superpoint+superglue'
    features = output_dir / 'features.h5'
    matches = output_dir / 'matches.h5'
    shutil.rmtree(input_dir, ignore_errors=True)  # clean output_root
    shutil.rmtree(output_root, ignore_errors=True)  # clean output_root
    os.makedirs(input_dir, exist_ok=True)
    os.makedirs(result_dir, exist_ok=True)
    os.makedirs(sfm_dir, exist_ok=True)

    # Image Preprocess
    print("image preprocessing ...")
    for file in tqdm(image_file_list):
        img_file = f"{data_root}/{file}"
        out_file = f"{input_dir}/{img_file.split('/')[-1]}"
        image = Image.open(img_file)
        image = image.resize([int(scale * x) for x in image.size])
        image_arr = np.asarray(image)
        if is_glare_remove:
            image_arr = remove_glare_v1(image_arr)
        image = Image.fromarray(image_arr)
        image.save(out_file)
        del image_arr, image

    # SfM try loop
    for num_matched in [10, 15]:
        model = None
        try:
            # Find image pairs via image retrieval
            image_list = [p.relative_to(input_dir).as_posix() for p in input_dir.iterdir()]
            retrieval_path = extract_features.main(retrieval_conf, input_dir, output_dir)
            pairs_from_retrieval.main(retrieval_path, sfm_pairs, num_matched=num_matched)
            # Extract and match local features
            feature_path = extract_features.main(feature_conf, input_dir, output_dir)
            match_path = match_features.main(matcher_conf, sfm_pairs, feature_conf['output'], output_dir)

            # 3D reconstruction
            model = reconstruction.main(sfm_dir, input_dir, sfm_pairs, feature_path, match_path, image_list=image_list, 
                                        camera_mode=pycolmap.CameraMode.SINGLE, 
                                        image_options=reader_options, 
                                        mapper_options={'ba_global_use_pba': True, 'ba_refine_principal_point': False})
            if model != None:
                break
        except:
            pass
    if model == None:
        print("[error] model is not exist")
        continue

    # visualization
    result_list = visualize_sfm_2d(model, input_dir, color_by='depth', n=1)
    for i, result in enumerate(result_list):
        if len(result) > 3:
            xyz = np.asarray(result[:-2])
            imgname = result[-1][0]
            # find image file and bbox
            for idx in indexes:
                image_file = df.loc[idx, 'image_file']
                if imgname in image_file:
                    bbox = df.loc[idx, 'bbox1']
                    bbox = np.array(bbox.split(':'), dtype='uint16')  # as int array
                    break
        else:  # result only xyz
            try:
                xyz = np.asarray([result[:3]])
                idx = indexes[i]
                image_file = df.loc[idx, 'image_file']
                imgname = image_file.split('/')[-1]
                bbox = df.loc[idx, 'bbox1']
                bbox = np.array(bbox.split(':'), dtype='uint16')  # as int array
            except:
                continue

        # define coor
        x0, y0, x1, y1 = bbox * scale
        cx = (x0 + x1) / 2
        cy = (y0 + y1) / 2
        new_w = (x1 - x0) * roi_ratio
        new_h = (y1 - y0) * roi_ratio
        new_x0 = cx - (new_w/2)
        new_y0 = cy - (new_h/2)
        new_x1 = cx + (new_w/2)
        new_y1 = cy + (new_h/2)
        roi = [new_x0, new_y0, new_x1, new_y1]

        # filter points in new region of interest
        filtered_points = []
        for x, y, z in xyz:
            if (new_x0 < x < new_x1)and(new_y0 < y < new_y1)and(60*scale < z):
                filtered_points.append([x, y, z])
        if len(filtered_points) == 0:
            print("[error] 0 filtered_points")
            continue
        filtered_points = np.array(filtered_points)
        depth_mean = np.median(filtered_points[:, 2])

        if is_find_left_right_points:
            # find left and right points
            i = np.argmin(filtered_points[:, 0])
            point_l = filtered_points[i, :2]
            i = np.argmax(filtered_points[:, 0])
            point_r = filtered_points[i, :2]
        else:
            # use bbox points
            point_l = np.array([new_x0, new_y0])
            point_r = np.array([new_x1, new_y1])

        # calculate realworld length
        project_lenth = np.linalg.norm(point_l - point_r)
        realworld_length = project_lenth / depth_mean * magnification
        df.loc[idx, "predict_size"] = realworld_length
        print(f"[overlap result] imgname point_l=({point_l[0]:.0f},{point_l[1]:.0f}) point_r=({point_r[0]:.0f},{point_r[1]:.0f}) depth={depth_mean:.2f} realworld_length={realworld_length:.2f}mm")

        # plot overlap result image
        try:
            if realworld_length > 1:
                image = Image.open(f"{input_dir}/{imgname}")
                dpi = 80
                fig, ax = plt.subplots(figsize=(image.width / dpi, image.height / dpi), dpi=dpi)
                ax.imshow(image)
                ax.add_patch(patches.Rectangle((x0, y0), x1-x0, y1-y0, linestyle=':', linewidth=1, edgecolor='#0000FF', facecolor="none"))  # add Rectangle
                ax.plot([point_l[0],point_r[0]], [point_l[1],point_r[1]], '.', color="#00FF00", linewidth=1)
                ax.plot([point_l[0],point_r[0]], [point_l[1],point_r[1]], '--', color="#00FF00", linewidth=1)
                ax.text(cx, y0, f"{realworld_length:.01f} mm", horizontalalignment='center', fontsize=14, backgroundcolor='none', c='#00FF00', fontstyle='normal')
                ax.set_axis_off()
                plt.savefig(f"{result_dir}/{imgname}", bbox_inches='tight', pad_inches=0)
                plt.close(fig)
                del image, fig, ax
        except:
            pass
        # break

    # copy result images
    shutil.copytree(result_dir, f"/home/data/Overlap_Result_PNG_1229_outdoor/{dir_name.split('/')[0]}", dirs_exist_ok=True)
    # save data frame
    df.to_csv(f"{data_root}/02_test_data_after_pred_bbox_back_pred_v0.2_1229.csv", index=False)
    print(f"[Done] ======================================== {dir_name} ==========================================")
    # break

[Start] ======================================== C_001/C_001_01 ==========================================
image preprocessing ...


100%|███████████████████████████████████████████| 50/50 [00:19<00:00,  2.54it/s]
[2022/12/29 08:27:19 hloc INFO] Extracting local features with configuration:
{'model': {'name': 'netvlad'},
 'output': 'global-feats-netvlad',
 'preprocessing': {'resize_max': 1024}}
[2022/12/29 08:27:19 hloc INFO] Found 50 images in root /home/test_outdoor/Image_Preprocessed.
100%|███████████████████████████████████████████| 50/50 [01:25<00:00,  1.71s/it]
[2022/12/29 08:28:56 hloc INFO] Finished exporting features.
[2022/12/29 08:28:56 hloc INFO] Extracting image pairs from a retrieval database.
[2022/12/29 08:28:57 hloc INFO] Found 500 pairs.
[2022/12/29 08:28:57 hloc INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 3},
 'output': 'feats-superpoint-n4096-r1024',
 'preprocessing': {'grayscale': True, 'resize_max': 1024}}
[2022/12/29 08:28:57 hloc INFO] Found 50 images in root /home/test_outdoor/Image_Preprocessed.


Loaded SuperPoint model


100%|███████████████████████████████████████████| 50/50 [00:31<00:00,  1.58it/s]
[2022/12/29 08:29:28 hloc INFO] Finished exporting features.
[2022/12/29 08:29:28 hloc INFO] Matching local features with configuration:
{'model': {'name': 'superglue',
           'sinkhorn_iterations': 50,
           'weights': 'outdoor'},
 'output': 'matches-superglue'}


Loaded SuperGlue model ("outdoor" weights)


...

100%|███████████████████████████████████████████| 50/50 [00:17<00:00,  2.81it/s]
[2022/12/29 08:50:28 hloc INFO] Extracting local features with configuration:
{'model': {'name': 'netvlad'},
 'output': 'global-feats-netvlad',
 'preprocessing': {'resize_max': 1024}}
[2022/12/29 08:50:28 hloc INFO] Found 50 images in root /home/test_outdoor/Image_Preprocessed.
100%|███████████████████████████████████████████| 50/50 [01:16<00:00,  1.52s/it]
[2022/12/29 08:51:53 hloc INFO] Finished exporting features.
[2022/12/29 08:51:53 hloc INFO] Extracting image pairs from a retrieval database.
[2022/12/29 08:51:53 hloc INFO] Found 500 pairs.
[2022/12/29 08:51:53 hloc INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 3},
 'output': 'feats-superpoint-n4096-r1024',
 'preprocessing': {'grayscale': True, 'resize_max': 1024}}
[2022/12/29 08:51:53 hloc INFO] Found 50 images in root /home/test_outdoor/Image_Preprocessed.


Loaded SuperPoint model


100%|███████████████████████████████████████████| 50/50 [00:33<00:00,  1.51it/s]
[2022/12/29 08:52:26 hloc INFO] Finished exporting features.
[2022/12/29 08:52:26 hloc INFO] Matching local features with configuration:
{'model': {'name': 'superglue',
           'sinkhorn_iterations': 50,
           'weights': 'outdoor'},
 'output': 'matches-superglue'}


Loaded SuperGlue model ("outdoor" weights)


 28%|███████████▋                              | 88/315 [01:15<02:53,  1.31it/s]

In [ ]:
for camera_id, camera in model.cameras.items():
    print(camera_id, camera)
    print(camera.params)

AttributeError: ignored

In [ ]:
#@title 7 statistics
df = pd.read_csv(f"{data_root}/02_test_data_after_pred_bbox_back_pred_v0.2.csv")
df['case'] = df['dir_name'].map(lambda x: x.split('/')[0])
df[~df['predict_size'].isnull()]

,image_file,bbox1,dir_name,predict_size,case
480,C_001/C_001_05/00006137.png,311:598:482:763,C_001/C_001_05,0.000000,C_001
482,C_001/C_001_05/00006139.png,312:601:484:768,C_001/C_001_05,0.000000,C_001
483,C_001/C_001_05/00006140.png,312:601:485:769,C_001/C_001_05,0.000000,C_001
484,C_001/C_001_05/00006141.png,312:600:484:772,C_001/C_001_05,0.000000,C_001
487,C_001/C_001_05/00006145.png,315:602:486:771,C_001/C_001_05,0.000000,C_001
...,...,...,...,...,...
5076,GI_003/GI_003_03/00005124.png,518:790:670:948,GI_003/GI_003_03,0.784759,GI_003
5077,GI_003/GI_003_03/00005139.png,846:465:1266:960,GI_003/GI_003_03,5.765220,GI_003
5078,GI_003/GI_003_03/00005141.png,841:471:1271:958,GI_003/GI_003_03,6.277679,GI_003
5080,GI_003/GI_003_03/00005150.png,505:817:689:959,GI_003/GI_003_03,0.962290,GI_003


In [ ]:
df.groupby(['case']).mean()

,predict_size
case,
C_001,3.014123
C_002,3.259552
C_003,4.286699
C_004_T colon polyp,6.191290
C_005_D colon polyp,4.199577
C_006_cecal seed,5.975812
GI_002,NaN
GI_003,1.467537
GI_004_duodenal polyp and pyloric ring,NaN


In [ ]:
df.groupby(['case']).std()

,predict_size
case,
C_001,2.607906
C_002,2.688285
C_003,3.733528
C_004_T colon polyp,2.655739
C_005_D colon polyp,2.641196
C_006_cecal seed,3.117097
GI_002,NaN
GI_003,0.931350
GI_004_duodenal polyp and pyloric ring,NaN
